In [25]:

import pandas as pd
import pyodbc
import time
import numpy as np
from datetime import datetime, date, time, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)


In [26]:
name_email = pd.read_csv('Local_Emails_Names.csv')
SM = pd.read_csv('Local_surveyMonkey.csv')
#Asana = pd.read_csv('asanaData.csv')





In [27]:
# First, make sure 'Staff Attending' is treated as a list (split by commas)
#Asana['Staff Attending'] = Asana['Staff Attending'].fillna('').apply(lambda x: [name.strip() for name in x.split(',') if name.strip()])

# Now explode the 'Staff Attending' column
#staff_expanded = Asana.explode('Staff Attending')

# Create the new column 'Attending AS' and fill with 'Staff'
#staff_expanded['Attending AS'] = 'Staff'



In [28]:
# reading in Asana data cleaned in previous file
use_cols = ['Name', 'Date', 'Staff Attending', 'FAB Attending',
            'Intern(s) Attending', 'Volunteers Attending','Event Status']

Asana = pd.read_csv('Local_asanaData.csv', usecols=use_cols)

# Renaming some columns
Asana.rename(columns={'Name': 'Event Name', 'Date': 'Event Date'}, inplace=True)



# making one row for each name in the staff, intern or Fab attending columns to be joined on 
# the survey monkey data and the Emails_Names to make the curvey completion check data frame. Had to use new pandas function
# melt. New to me, but looks like it works. The melting is liek unpacking but let me classify thier "Role"
 # at the same time
    
cols_to_melt = ['Staff Attending', 'FAB Attending', 'Intern(s) Attending', 'Volunteers Attending']


# Drop rows where all attending fields are empty or NaN
Asana.dropna(subset=cols_to_melt, how='all', inplace=True)


role_map = {
    'Staff Attending': 'Staff',
    'FAB Attending': 'FAB',
    'Intern(s) Attending': 'Intern',
    'Volunteers Attending': 'Volunteer'
}

melted = Asana.melt(
    id_vars=[col for col in Asana.columns if col not in cols_to_melt], 
    value_vars=cols_to_melt, 
    var_name='Role', 
    value_name='Attending Email'
)

# Drop rows where the melted value is NaN or empty string
melted = melted[melted['Attending Email'].notna() & (melted['Attending Email'].str.strip() != '')]


#careful with split it needs soo many checks
def split_emails(x):
    if not isinstance(x, str) or x.strip() == '':
        return []
    return [name.strip() for name in x.split(',') if name.strip()]

melted['Attending Email'] = melted['Attending Email'].fillna('').apply(split_emails)
melted = melted.explode('Attending Email')

melted['Attending As'] = melted['Role'].map(role_map)
melted = melted.drop(columns=['Role'])

In [29]:
melted.head(100)


,Event Name,Event Date,Event Status,Attending Email,Attending As
0,"Sparta Books, Blocks, and Balls",2025-09-20,Registered & Preparing,ltagle@familyfutures.org,Staff
0,"Sparta Books, Blocks, and Balls",2025-09-20,Registered & Preparing,salonso@familyfutures.org,Staff
1,Sheldon WIC Clinic,2025-09-22,N/A - Tabling,salonso@familyfutures.org,Staff
2,Sheldon WIC Clinic,2025-10-03,N/A - Tabling,salonso@familyfutures.org,Staff
3,Sheldon WIC Clinic,2025-09-19,N/A - Tabling,salonso@familyfutures.org,Staff
4,Sheldon WIC Clinic,2025-09-09,N/A - Tabling,salonso@familyfutures.org,Staff
5,Sheldon WIC Clinic,2025-09-09,N/A - Tabling,salonso@familyfutures.org,Staff
6,Sheldon WIC Clinic,2025-08-27,N/A - Tabling,salonso@familyfutures.org,Staff
7,Sheldon WIC Clinic,2025-08-22,N/A - Tabling,salonso@familyfutures.org,Staff
8,Sheldon WIC Clinic,2025-10-28,N/A - Tabling,salonso@familyfutures.org,Staff


In [30]:
name_email.columns



Index(['Display name', 'Department', 'Username', 'Email'], dtype='object')

In [31]:
SM.columns

Index(['First Name', 'Last Name', 'Email', 'Role', 'Event/Tabling',
       'Number Kids Engaged', 'Number Adults Engaged', 'Nursing', 'Changing',
       'NN Adult Engaged', 'NN Kids Engaged', 'Both', 'Clicker?',
       'Clicker Other', 'Networked Orgs?', 'Names of Networked Orgs',
       'Activities/Materials Feedback', 'Recommend Again?',
       'Recommendation Reason', 'Mission Moment', 'Further Comments',
       'Event Date', 'Event Name', 'Lead Assignee', 'Person Name'],
      dtype='object')

In [32]:
#making a library of typos on the old name column for now. hope to clean data and submit to SQL , but I guess this might always need to be here. 
# Dictionary of replacements
name_map = {
    'Leticia': 'Leticia Miranda',
    'Gabi Nye': 'Gabrielle Nye',
    'Susan Alonso': 'Susi Alonso',
    'Karissa': 'Karissa Monzo',
    'Kayla Hopkins':'Kayla Smith â€“ Hopkins',
    'Cam Simon':'Cameron Simon'
}

# Apply to the 'Person Name' column
SM['Person Name'] = SM['Person Name'].replace(name_map)





In [33]:
melted.columns

Index(['Event Name', 'Event Date', 'Event Status', 'Attending Email',
       'Attending As'],
      dtype='object')

In [34]:
melted = melted.merge(name_email, how='left', left_on='Attending Email', right_on='Email')

In [35]:
melted.columns

Index(['Event Name', 'Event Date', 'Event Status', 'Attending Email',
       'Attending As', 'Display name', 'Department', 'Username', 'Email'],
      dtype='object')

In [36]:
SM.rename(columns={'Event Name': 'Event SM', 'Event Date': 'Date SM', 'Email': 'Email SM'}, inplace=True)

In [37]:


#melted = melted.merge(SM, how='outer', left_on=['Display name','Event Date' ], right_on=['Person Name','Date SM'])


# 1. Standardize names and dates before merge
melted['Display name'] = melted['Display name'].astype(str).str.strip().str.lower()
melted['Event Date'] = pd.to_datetime(melted['Event Date'], errors='coerce')

SM['Person Name'] = SM['Person Name'].astype(str).str.strip().str.lower()
SM['Date SM'] = pd.to_datetime(SM['Date SM'], errors='coerce')

#  Merge with full outer join
merged = melted.merge(
    SM,
    how='outer',
    left_on=['Display name', 'Event Date'],
    right_on=['Person Name', 'Date SM']
)


# Remove rows where "test" appears in Person Name
merged = merged[~merged['Person Name'].str.contains('test', na=False)] 


# Restore title case for reporting
merged['Display name'] = merged['Display name'].str.title()
merged['Person Name'] = merged['Person Name'].str.title()
merged['Attending Email'] = merged['Attending Email'].str.title()







In [38]:
# see what's not matching
#missing = merged[merged['Person Name'].isna() | merged['Display name'].isna()]
#missing.head(25)

In [39]:
melted.shape

(220, 9)

In [40]:
merged.columns

Index(['Event Name', 'Event Date', 'Event Status', 'Attending Email',
       'Attending As', 'Display name', 'Department', 'Username', 'Email',
       'First Name', 'Last Name', 'Email SM', 'Role', 'Event/Tabling',
       'Number Kids Engaged', 'Number Adults Engaged', 'Nursing', 'Changing',
       'NN Adult Engaged', 'NN Kids Engaged', 'Both', 'Clicker?',
       'Clicker Other', 'Networked Orgs?', 'Names of Networked Orgs',
       'Activities/Materials Feedback', 'Recommend Again?',
       'Recommendation Reason', 'Mission Moment', 'Further Comments',
       'Date SM', 'Event SM', 'Lead Assignee', 'Person Name'],
      dtype='object')

In [41]:
#for iloc[where 'Role'='Volunteer']then 'Event Name'= 'Event SM'
merged.loc[merged['Role'] == 'Volunteer', 'Event Name'] = 'Event SM'

In [42]:
merged.to_csv('Local_SurveyCompletions.csv')
